In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json

from skimage.io import imread, imsave
from skimage.measure import find_contours
from skimage.filters import gaussian
from skimage.segmentation import active_contour
from skimage.morphology import binary_erosion
from skimage.draw import polygon
from skimage.transform import warp_polar
from skimage.transform import warp, EuclideanTransform

from scipy.ndimage import distance_transform_edt
from scipy.optimize import fmin, least_squares
from scipy.signal import savgol_filter
from scipy.io import savemat

In [2]:
def get_params_for_date(microscope, date, metadata):
    """Retrieve all parameters for a given microscope and date."""
    try:
        return metadata[microscope][date]
    except KeyError:
        print("Data not found for the given microscope and date.")

In [3]:
with open('../metadata.json') as f:
    metadata = json.load(f)

In [5]:
# Example usage
scope_name = 'Tweez scope'
exp_date = '2023_11_15'
colonies = get_params_for_date(scope_name, exp_date, metadata)

result = get_params_for_date('Tweez scope', '2023_11_15', metadata)
result

{'0': {'cx': 524, 'cy': 480, 'radius': 100, 'radj': 50},
 '1': {'cx': 505, 'cy': 520, 'radius': 100, 'radj': 50},
 '3': {'cx': 510, 'cy': 510, 'radius': 100, 'radj': 50}}

In [11]:
def make_video(images_folder,  video_name):
    """
    Makes a video from a sequence of png images.
    
    Parameters:
    - images_folder (string): Folder that contains the png files.
    - word_cont (string): Word that the files contain.
    - video_name (string): Name of the output file.

    Returns:
    - 
    """
    images = [img for img in os.listdir(images_folder) if img.endswith(".png")]
    images.sort()
    frame = cv2.imread(os.path.join(images_folder, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name, 0, 7, (width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(images_folder, image)))

    video.release()

In [7]:
##################
# analysis params
##################

#folder = '/home/guillermo/Microscopy'
folder = '/mnt/ff9e5a34-3696-46e4-8fa8-0171539135be'
#scope_name = 'Ti scope'
scope_name = 'Tweez scope'
path_scope = os.path.join(folder, scope_name)
exp_date = '2023_12_08'
path = os.path.join(path_scope, exp_date)
folder_masks = 'contour_masks'
folder_results = 'results'
folder_fluo = 'fluo'
folder_graphs = 'graphs'
pos = 0
#fname = f'{exp_date}_10x_1.0x_pLPT20&41_single_TiTweez_Pos{pos}.ome.tif'
#fname_mask = 'mask_' + fname

### params for repressilator single reporter
yfp_chn = 0
cfp_chn = 1
ph_chn = 2
fluo_chns = 2

In [14]:
poss = [2,4,5,6,7,11,12,13,17,18,19,21,22,23,24,29,30,31,32,34,35]

In [15]:
for pos in poss:
    fname = f'{exp_date}_10x_1.0x_MC4100_pLPT20&41_TiTweez_Pos{pos}.ome.tif'
    fname_mask = 'mask_' + fname
    path_mask = os.path.join(path, folder_masks, fname_mask)
    #print(path_mask)
    mask = imread(path_mask)
    mask = mask[:216]
    imsave(path_mask, mask)
    make_video(os.path.join(path, folder_masks,f"temp_pos{pos}"), 
           os.path.join(path, folder_masks,f"contour_pos{pos}.avi"))